In [8]:
import cProfile
import io
import pstats

import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial import cKDTree
from scipy.interpolate import griddata

import functions


In [2]:
import laspy
import numpy as np

def extract_vegetation_points(LasData, ndvi_threshold=0.1):
    """
    Extract vegetation points based on classification and NDVI threshold.
    
    Parameters:
    - LasData (laspy.LasData): Input point cloud data.
    - ndvi_threshold (float): The NDVI threshold to classify vegetation.
    
    Returns:
    - veg_points (laspy.LasData): Filtered vegetation points as a new LasData object.
    """
    
    # Filter points based on classification (vegetation-related classes)
    possible_vegetation_points = LasData[(LasData.classification == 1) |  # Unclassified
                                         (LasData.classification == 3) |  # Low vegetation
                                         (LasData.classification == 4) |  # Medium vegetation
                                         (LasData.classification == 5)]   # High vegetation

    # Calculate NDVI
    red = possible_vegetation_points.red
    nir = possible_vegetation_points.nir
    ndvi = (nir.astype(float) - red) / (nir + red)

    # Filter the points whose NDVI is greater than the threshold
    veg_points = possible_vegetation_points[ndvi > ndvi_threshold]

    return veg_points

def save_vegetation_points_as_las(LasData, output_file, veg_points, ndvi_threshold=0.1):
    """
    Extract vegetation points and save them as a new LAS file.
    
    Parameters:
    - LasData (laspy.LasData): Input point cloud data.
    - output_file (str): Path to the output LAS file.
    - ndvi_threshold (float): NDVI threshold for vegetation points.
    
    Returns:
    - None
    """
    # Extract vegetation points
    
    # Check if any points were filtered
    if len(veg_points) == 0:
        print("No vegetation points found with the given NDVI threshold.")
        return
    
    # Create a new LasData object for the vegetation points
    veg_las_data = laspy.LasData(LasData.header)
    veg_las_data.points = veg_points.points  # Assign the filtered vegetation points

    # Save the vegetation points to a new LAS file
    with laspy.open(output_file, mode="w", header=veg_las_data.header) as writer:
        writer.write_points(veg_las_data.points)
    
    print(f"Vegetation points saved to {output_file}")


# Example usage:
# Load the LAS/LAZ file
file_path = "../data/25DN2_10.LAZ"
output_las = "../data/output_vegetation.las"



In [3]:
with laspy.open(file_path, laz_backend=laspy.LazBackend.Lazrs) as las_file:
    las_data = las_file.read()

veg_points = extract_vegetation_points(las_data, ndvi_threshold=0.1)

In [29]:
def vegetation_raster_idw(LasData, points, resolution, search_radius=1, power=2):
    """
    Create a vegetation raster using Inverse Distance Weighting (IDW) interpolation.

    Parameters:
    - LasData (laspy.LasData): Input LAS data.
    - points (laspy.LasData): Vegetation points to be interpolated.
    - center_point (tuple): Center coordinates (x, y) of the region of interest.
    - resolution (float): Resolution of the raster.
    - half_size (float): Half of the size of the tile (defines the extent of the raster).
    - search_radius (float): Radius for the IDW search. Default is 1.
    - power (float): Power parameter for the IDW algorithm. Default is 2.

    Returns:
    - vege_raster (np.ndarray): Generated raster for vegetation.
    - grid_center_xy (tuple): Grid of x, y center coordinates for each raster cell.
    """

    # Get the extents of the point cloud
    min_x, max_x = round(LasData.x.min()), round(LasData.x.max())
    min_y, max_y = round(LasData.y.min()), round(LasData.y.max())
    
    # Define the size of the region (half_size defines the full size as 2 * half_size)
    x_length = max_x - min_x
    y_length = max_y - min_y
    
    # Number of rows and columns based on the extents and resolution
    cols = round(x_length / resolution)
    rows = round(y_length / resolution)
    
    # Initialize the raster grid with NaN values (no data value)
    vege_raster = np.full((rows, cols), np.nan, dtype=np.float32)

    # Calculate the center coordinates for each grid cell
    grid_center_xy = functions.raster_center_coords(min_x, max_x, min_y, max_y, resolution)

    if len(points) == 0:
        print("step4, vegetation_raster_idw: There are no vegetation points in the current area.")
        return vege_raster, grid_center_xy

    # Convert the LAS points to a NumPy array for processing
    points_list = np.array(points.xyz)  # Transform the laspy point cloud to numpy 2D array.

    # Perform IDW interpolation using k-d tree
    functions.interpolation_idw_kdtree(points_list, vege_raster, grid_center_xy, search_radius, power)

    return vege_raster, grid_center_xy

In [5]:
save_vegetation_points_as_las(las_data, output_las, veg_points, ndvi_threshold=0.1)

Vegetation points saved to ../data/output_vegetation.las


In [33]:
def step4_main(LasData, veg_points, resolution=0.5, search_radius=1.0, idw_power=2.0, write_output=True, show_fig=False):

    vege_points = extract_vegetation_points(LasData)
    vegetation_data = vegetation_raster_idw(LasData, veg_points, resolution,
                                             search_radius, idw_power)
    veg_raster = vegetation_data[0]
    grid_centers = vegetation_data[1]
    top_left_x = grid_centers[0][0, 0] - resolution / 2
    top_left_y = grid_centers[1][0, 0] + resolution / 2
    if write_output:
        transform = functions.create_affine_transform(top_left_x, top_left_y, resolution)
        functions.write_output(vege_points, veg_raster, transform, "../data/chm.tiff")

In [32]:
step4_main(las_data, veg_points, resolution=0.5, search_radius=1.0, idw_power=2.0, write_output=True, show_fig=False)

PicklingError: Could not pickle the task to send it to the workers.